# Preprocessing

In [2]:
# import libraries 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [3]:
# import the dataset 
dataset2008 = pd.read_excel( 'all_alpha_08.xls' ) 
dataset2009 = pd.read_excel( 'all_alpha_09.xls' ) 
dataset2010 = pd.read_excel( 'all_alpha_10.xls' ) 
dataset2011 = pd.read_excel( 'all_alpha_11.xlsx' ) 
dataset2012 = pd.read_excel( 'all_alpha_12.xlsx' ) 
dataset2013 = pd.read_excel( 'all_alpha_13.xlsx' ) 
dataset2014 = pd.read_excel( 'all_alpha_14.xlsx' ) 
dataset2015 = pd.read_excel( 'all_alpha_15.xlsx' ) 
dataset2016 = pd.read_excel( 'all_alpha_16.xlsx' ) 
dataset2017 = pd.read_excel( 'all_alpha_17.xlsx' ) 
dataset2018 = pd.read_excel( 'all_alpha_18.xlsx' )

In [4]:
# Shape of datasets 
print( '2008 ->', dataset2008.shape)
print( '2009 ->', dataset2009.shape)
print( '2010 ->', dataset2010.shape)
print( '2011 ->', dataset2011.shape)
print( '2012 ->', dataset2012.shape)
print( '2013 ->', dataset2013.shape)
print( '2014 ->', dataset2014.shape)
print( '2015 ->', dataset2015.shape)
print( '2016 ->', dataset2016.shape)
print( '2017 ->', dataset2017.shape)
print( '2018 ->', dataset2018.shape)


2008 -> (2404, 18)
2009 -> (2561, 17)
2010 -> (2326, 17)
2011 -> (2171, 17)
2012 -> (2252, 17)
2013 -> (2397, 18)
2014 -> (2517, 18)
2015 -> (2653, 18)
2016 -> (2589, 18)
2017 -> (2589, 18)
2018 -> (2713, 18)


In [5]:
# Common columns across the datasets and their datatype.

# 'Model'                - string
# 'Displ'                - float
# 'Cyl'                  - float
# 'Trans'                - string
# 'Drive'                - string
# 'Fuel '                - string
# 'Stnd'                 - string
# 'Underhood ID'         - string
# 'Veh Class'            - string
# 'Air Pollution Score'  - float
# 'City MPG'             - float
# 'Hwy MPG'              - float
# 'Cmb MPG'              - float
# 'Greenhouse Gas Score' - float
# 'SmartWay'             - string
# 'Year'                 - int


## Cleaning

Here, we clean all 11 datasets and combine all to get a master dataset. It is done by defining a function which will clean the datasets. Since all 11 datasets are similar, one function can be used to clean all.


In [7]:
# Define a function to clean the datasets 
def function_clean_data(dataset, year):

    # Create copy of the dataset 
    dataset_nonull = dataset.copy()

    # Select the required columns 
    dataset_nonull = dataset_nonull.loc[:, ['Model', 'Displ', 'Cyl', 'Trans', 'Drive', 'Fuel', 'Stnd', 'Underhood ID', 'Veh Class', 'Air Pollution Score', 'City MPG', 'Hwy MPG', 'Cmb MPG', 'Greenhouse Gas Score' , 'SmartWay' ]]

    # Add a column Year and value as year to identify the dataset year 
    dataset_nonull[ 'Year' ] = year

    # Drop duplicates from the dataset 
    dataset_nonull.drop_duplicates(keep='first', inplace=True)

    # Drop Null values 
    dataset_nonull = dataset_nonull.drop(dataset_nonull[dataset_nonull.isnull().any(axis=1)].index, axis=0)

    # In dataset2008, number of cylinders is string eg: '(6 cyl)', convert it to float
    if (year == 2008):
        dataset_nonull.loc[:, 'Cyl'] = dataset_nonull.loc[ : , 'Cyl'].str[1: -5].astype( 'float64' )

    # In some datasets some values are marked as MOD - which we will remove.
    dataset_nonull = dataset_nonull.drop(dataset_nonull[ (dataset_nonull == 'Mod').any(axis = 1)].index, axis=0)

    # Create a new dataframe for formatteddataset 
    dataset_formatted = dataset_nonull.copy()

    # Here, we will format all 11 datasets to have the same datatype and consistent values 
    splitted_list2 = []
    for i in dataset_nonull[dataset_nonull.loc[:, 'Fuel'].str.contains('/')].index: 
        record = dataset_nonull.loc[i, :] 
        record1 = [record[0], record[1], record[2], record[3], record[4], str(record[5]).split( '/' ) [0], record[6], record[7], record[8], str(record[9]).split( '/')[0], str(record[10]).split( '/')[0], str(record[11]).split( '/' ) [0], str(record[12]).split('/')[0], str(record[13]).split('/')[0], record[14], record[15]] 
        record2 = [record[0], record[1], record[2], record[3], record[4], str(record[5]).split( '/')[len(str(record[5]).split( '/'))-1], record[6], record[7], record[8], str(record[9]).split( '/')[len(str(record[9]).split( '/'))-1], str(record[10]).split('/')[len(str(record[10]).split( '/'))-1], str(record[11]).split( '/')[len(str(record[11]).split( '/'))-1], str(record[12]).split('/')[len(str(record[12]).split( '/'))-1], str(record[13]).split( '/')[len(str(record[13]).split('/'))-1], record[14], record[15]] 
        splitted_list2.append(record1) 
        splitted_list2.append(record2) 
    dataset_formatted = dataset_nonull.copy() 
    dataset_formatted = dataset_formatted.drop(dataset_nonull[dataset_nonull.loc[:, 'Fuel'].str.contains( '/')].index, axis=0) 
    dataset_formatted = dataset_formatted.append(pd.DataFrame(splitted_list2, columns = dataset_nonull.columns) )

    # Standardise the datatype of the columns
    dataset_formatted.loc[:, 'Displ'] = dataset_formatted.loc[ :, 'Displ'].astype( 'float64' ) 
    dataset_formatted.loc[ :, 'Cyl'] = dataset_formatted.loc[ : , 'Cyl'].astype( 'float64') 
    dataset_formatted.loc[:, 'Air Pollution Score'] = dataset_formatted.loc[ :, 'Air Pollution Score'].astype( 'float64' ) 
    dataset_formatted.loc[ :, 'City MPG'] = dataset_formatted.loc[ :, 'City MPG' ].astype( 'float64' ) 
    dataset_formatted.loc[ :, 'Hwy MPG' ] = dataset_formatted.loc[ : , 'Hwy MPG' ].astype( 'float64' ) 
    dataset_formatted.loc[ :, 'Cmb MPG' ] = dataset_formatted.loc[ : , 'Cmb MPG' ].astype( 'float64' ) 
    dataset_formatted.loc[ :, 'Greenhouse Gas Score'] = dataset_formatted.loc[ : , 'Greenhouse Gas Score'].astype( 'float64')

    # Indexing of the dataset is corrected after droppping and cleaning is done 
    dataset_formatted.reset_index(inplace = True, drop = True)

    return dataset_formatted


In [8]:
# We run the cleaning function on all 11 datasets 
dataset_list = [dataset2008, dataset2009, dataset2010, dataset2011, dataset2012, dataset2013, dataset2014, dataset2015, dataset2016, dataset2017, dataset2018] 
year_list = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018] 
formatted_dataset_list = [] 
for i in range(0,11) :
    formatted_dataset = function_clean_data(dataset_list[i], year_list[i]) 
    formatted_dataset_list.append(formatted_dataset)


In [9]:
# Merge all the cleaned datasets into one master dataset to do analysis 
master_dataset = formatted_dataset_list[0] 
for i in range(1,11):
    master_dataset = master_dataset.append(formatted_dataset_list[i]) 
master_dataset.reset_index(inplace = True, drop = True)


In [12]:
print('Shape of the master dataset:')
print(master_dataset.shape)
print('----------------------------------------------------')
print('Dataset details')
display(master_dataset.describe())
print('----------------------------------------------------')
print('Dataset info')
print(master_dataset.info())

Shape of the master dataset:
(26208, 16)
----------------------------------------------------
Dataset details


,Displ,Cyl,Air Pollution Score,City MPG,Hwy MPG,Cmb MPG,Greenhouse Gas Score,Year
count,26208.000000,26208.000000,26208.000000,26208.000000,26208.000000,26208.000000,26208.000000,26208.000000
mean,3.324531,5.800748,5.773237,19.659646,26.677999,22.210966,4.906174,2013.159570
std,1.359056,1.885750,1.346258,8.188357,7.922897,7.958430,1.972667,3.200304
min,0.600000,2.000000,1.000000,7.000000,11.000000,9.000000,0.000000,2008.000000
25%,2.000000,4.000000,5.000000,15.000000,22.000000,18.000000,4.000000,2010.000000
50%,3.000000,6.000000,6.000000,18.000000,26.000000,21.000000,5.000000,2013.000000
75%,4.000000,6.000000,6.000000,22.000000,30.000000,25.000000,6.000000,2016.000000
max,8.400000,16.000000,9.500000,145.000000,121.000000,133.000000,10.000000,2018.000000


----------------------------------------------------
Dataset info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26208 entries, 0 to 26207
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Model                 26208 non-null  object 
 1   Displ                 26208 non-null  float64
 2   Cyl                   26208 non-null  float64
 3   Trans                 26208 non-null  object 
 4   Drive                 26208 non-null  object 
 5   Fuel                  26208 non-null  object 
 6   Stnd                  26208 non-null  object 
 7   Underhood ID          26208 non-null  object 
 8   Veh Class             26208 non-null  object 
 9   Air Pollution Score   26208 non-null  float64
 10  City MPG              26208 non-null  float64
 11  Hwy MPG               26208 non-null  float64
 12  Cmb MPG               26208 non-null  float64
 13  Greenhouse Gas Score  26208 non-null  float64
 14  Smar

In [14]:
# Check for incongruenciaes in values 
print('Distinct values in Fuel column')
print(master_dataset.loc[ :, 'Fuel' ].value_counts()) 
print('-----------------------------------------------------')
print('Distinct values in SmartWay column')
print(master_dataset. loc[ :, 'SmartWay' ].value_counts())
print('-----------------------------------------------------')
print('There are some incongruencies in the data here such as typo and casing issues.')

# Fix the typo and casing 
# Fuel column 
master_dataset.loc[:, 'Fuel'].replace(to_replace = 'ethanol', value = 'Ethanol', inplace = True ) 
master_dataset.loc[ :, 'Fuel'].replace(to_replace ='gas', value ='Gas' , inplace = True ) 
master_dataset.loc[:, 'Fuel'].replace(to_replace = 'Electricty' , value ='Electricity', inplace = True ) 
master_dataset.loc[:,'Fuel'].replace(to_replace= 'diesel', value = 'Diesel', inplace = True )

#SmartWay column 
master_dataset.loc[:, 'SmartWay'].replace(to_replace = 'no', value = 'No', inplace = True )
master_dataset.loc[:, 'SmartWay'].replace(to_replace = 'yes', value ='Yes', inplace = True )

print('Distinct values in Fuel column after fixing typo and casing issues')
print('-----------------------------------------------------')
print(master_dataset.loc[ :, 'Fuel' ].value_counts()) 
print('-----------------------------------------------------')
print(master_dataset. loc[ :, 'SmartWay' ].value_counts())

Distinct values in Fuel column
Gasoline       22993
Ethanol         1397
Gas             1393
Electricity      207
Diesel           192
CNG               26
Name: Fuel, dtype: int64
-----------------------------------------------------
Distinct values in SmartWay column
No       20534
Yes       5497
Elite      177
Name: SmartWay, dtype: int64
-----------------------------------------------------
There are some incongruencies in the data here such as typo and casing issues.
Distinct values in Fuel column after fixing typo and casing issues
-----------------------------------------------------
Gasoline       22993
Ethanol         1397
Gas             1393
Electricity      207
Diesel           192
CNG               26
Name: Fuel, dtype: int64
-----------------------------------------------------
No       20534
Yes       5497
Elite      177
Name: SmartWay, dtype: int64


In [11]:
display(master_dataset.head())

,Model,Displ,Cyl,Trans,Drive,Fuel,Stnd,Underhood ID,Veh Class,Air Pollution Score,City MPG,Hwy MPG,Cmb MPG,Greenhouse Gas Score,SmartWay,Year
0,ACURA MDX,3.7,6.0,Auto-S5,4WD,Gasoline,U2,8HNXT03.7PKR,SUV,7.0,15.0,20.0,17.0,4.0,No,2008
1,ACURA MDX,3.7,6.0,Auto-S5,4WD,Gasoline,B5,8HNXT03.7PKR,SUV,6.0,15.0,20.0,17.0,4.0,No,2008
2,ACURA RDX,2.3,4.0,Auto-S5,4WD,Gasoline,U2,8HNXT02.3DKR,SUV,7.0,17.0,22.0,19.0,5.0,No,2008
3,ACURA RDX,2.3,4.0,Auto-S5,4WD,Gasoline,B5,8HNXT02.3DKR,SUV,6.0,17.0,22.0,19.0,5.0,No,2008
4,ACURA RL,3.5,6.0,Auto-S5,4WD,Gasoline,U2,8HNXV03.5HKR,midsize car,7.0,16.0,24.0,19.0,5.0,No,2008
